In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
import numpy as np
from tensorflow.keras.preprocessing.sequence import pad_sequences
import pickle as cPickle
import pandas as pd
from keras.callbacks import EarlyStopping
from tensorflow.keras.utils import to_categorical
from sklearn.preprocessing import LabelBinarizer
from keras.layers import LSTM, Dense, TimeDistributed, Bidirectional
import sklearn.metrics
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from keras.models import Sequential
from keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras import Model
from keras.layers import Dense,Embedding,LSTM,Dropout,Bidirectional
from keras import regularizers
from keras import models

In [ ]:
path_data = "/content/drive/MyDrive/SuccKD_MultiSpecies/Data/"
path_model = "/content/drive/MyDrive/SuccKD_MultiSpecies/Model/"
path_result = "/content/drive/MyDrive/SuccKD_MultiSpecies/Result_cv/"

In [ ]:
file_train = "Train_Generic_succ.csv"

In [ ]:
# Build an amino acid dictionary
def twoTupleDic1():
    AA_list_sort = ['G','A','V','L','I','M','P','F','W','S','T','N','Q','Y','C','K','R','H','D','E','X']

    AA_dict = {}
    numm = 1
    for i in AA_list_sort:
        AA_dict[i] = numm
        numm += 1
    return AA_dict
def ProSentence(pro, K):
	sentence = ""
	length = len(pro)
	for i in range(length - K + 1):
		sentence += pro[i: i + K] + " "
    #delete extra space
	sentence = sentence[0 : len(sentence) - 1]
	return sentence
k =1#1-gram
word_index1 = twoTupleDic1()
vocab_size = len(word_index1)


In [ ]:
#  Load train data and pre-processing
df_train =pd.read_csv(path_data+file_train, delimiter= ',')
texts_train =[]
for i in df_train['Peptide_WS_33']:
  temp = ProSentence(i,k)
  texts_train.append(temp)
df_train['k_mer'] =texts_train
train_sequences = []
for each in texts_train:
    each_index_list = []
    each = each.split(' ')
    for i in each:
        each_index_list.append(word_index1[i])
    train_sequences.append(each_index_list)
# Tokenizer
data_token = []
for i in df_train['k_mer']:
   data_token.append(i.split())
MAX_SEQUENCE_LENGTH = len(data_token[1])
Xtrain = pad_sequences(train_sequences, maxlen=MAX_SEQUENCE_LENGTH)
ytrain = np.array(df_train['Label'])
ytrain = np.array(ytrain)
#one-hot encoding on the labels
lb = LabelBinarizer()
ytrain= lb.fit_transform(ytrain)
ytrain = to_categorical(ytrain)
ytrain.shape


(14250, 2)

In [ ]:
TIME_STEPS = MAX_SEQUENCE_LENGTH
INPUT_SIZE = 300

In [ ]:
# KD2_succi
import tensorflow as tf
from tensorflow.keras import optimizers
from tensorflow.keras import models, layers, losses

# Teacher 1
def create_cnn_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Dropout(0.4))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.4))
    model.add(layers.BatchNormalization())
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu'))
    model.add(layers.MaxPooling1D(pool_size=2))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model

# Teacher 2
def create_bilstm_model():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.BatchNormalization())
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
# Student
def create_student():
    model = models.Sequential()
    model.add(layers.Embedding(vocab_size+1, 300, input_length=MAX_SEQUENCE_LENGTH,trainable=True))
    model.add(layers.Conv1D(filters=32, kernel_size=3, activation='relu', padding='same'))
    model.add(layers.MaxPooling1D(2))
    model.add(layers.Dropout(0.2))
    model.add(layers.Bidirectional(layers.LSTM(units=32, return_sequences=True), input_shape=(TIME_STEPS, INPUT_SIZE)))
    model.add(layers.Flatten())
    model.add(layers.Dropout(0.4))
    model.add(layers.Dense(64, activation='relu'))
    model.add(layers.Dense(2, activation='softmax'))  # Binary classification (softmax for distillation)
    return model
#  KD2_succi: Knowlege distillation
import tensorflow as tf
from tensorflow.keras import optimizers

class Distiller(tf.keras.Model):
    def __init__(self, teacher1, teacher2, student, alpha=0.5, temperature=3.0):
        super(Distiller, self).__init__()
        self.teacher1 = teacher1
        self.teacher2 = teacher2
        self.student = student
        self.alpha = alpha
        self.temperature = temperature

        # Optimizer for the student
        self.student_optimizer = AdamW(learning_rate=0.0001)

    def compile(self, metrics, student_loss_fn, distillation_loss_fn):
        super(Distiller, self).compile(metrics=metrics)
        self.student_loss_fn = student_loss_fn
        self.distillation_loss_fn = distillation_loss_fn

    def call(self, inputs):
        """
        This method defines how the model processes inputs.
        Args:
        - inputs: Input data (Xtest_student or any other data).

        Returns:
        - Output logits from the student model.
        """
        student_logits = self.student(inputs, training=False)
        return student_logits

    def train_step(self, data):
        x, y = data

        # Sequentially forward pass through teachers
        teacher1_logits = self.teacher1(x, training=False)
        teacher2_logits = self.teacher2(x, training=False)

        with tf.GradientTape() as tape:
            # Forward pass through student
            student_logits = self.student(x, training=True)

            # Compute the task loss (for hard labels)
            student_loss = self.student_loss_fn(y, student_logits)

            # Compute distillation losses (for soft labels)
            distill_loss1 = self.distillation_loss_fn(
                tf.nn.softmax(teacher1_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            distill_loss2 = self.distillation_loss_fn(
                tf.nn.softmax(teacher2_logits / self.temperature, axis=1),
                tf.nn.softmax(student_logits / self.temperature, axis=1)
            ) * (self.temperature ** 2)

            # Combined loss
            total_loss = self.alpha * student_loss + (1 - self.alpha) * (distill_loss1 + distill_loss2)

        # Compute gradients and update student model
        grads = tape.gradient(total_loss, self.student.trainable_variables)
        self.student_optimizer.apply_gradients(zip(grads, self.student.trainable_variables))

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        # Return results
        results = {
            'student_loss': student_loss,
            'distill_loss1': distill_loss1,
            'distill_loss2': distill_loss2
        }
        results.update({m.name: m.result() for m in self.metrics})
        return results

    def test_step(self, data):
        x, y = data
        student_logits = self.student(x, training=False)

        # Compute the task loss (for hard labels)
        student_loss = self.student_loss_fn(y, student_logits)

        # Update metrics for the student
        self.compiled_metrics.update_state(y, student_logits)

        return {
            'student_loss': student_loss,
            **{m.name: m.result() for m in self.metrics}
        }



In [ ]:
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import KFold
from tensorflow.keras.optimizers import AdamW
k = 10
kf = KFold(n_splits=k, shuffle=True, random_state=42)
all_metrics = []
confusion_matrices = []
# K-fold cross-validation
for fold, (train_index, val_index) in enumerate(kf.split(Xtrain)):
    print(f"Training on fold {fold + 1}/{k}...")
    X_train_fold, X_val_fold = Xtrain[train_index], Xtrain[val_index]
    y_train_fold, y_val_fold = ytrain[train_index], ytrain[val_index]
    teacher1_fold = create_cnn_model()
    teacher2_fold = create_bilstm_model()
    student_fold = create_student()
    teacher1_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher2_fold.compile(
        loss=keras.losses.CategoricalCrossentropy(from_logits=True),
        metrics=[
            "categorical_accuracy",
            "AUC",
        ],
        optimizer=keras.optimizers.AdamW(learning_rate=0.0001),
    )
    teacher1_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    teacher2_fold.fit(
        X_train_fold, y_train_fold,
        batch_size =16,
        epochs=50
    )

    # Re-initialize the distiller for each fold
    distiller_fold = Distiller(teacher1=teacher1_fold, teacher2=teacher2_fold, student=student_fold, alpha=0.5, temperature=10)
    distiller_fold.compile(
        student_loss_fn=keras.losses.CategoricalCrossentropy(from_logits=True),

        metrics=["categorical_accuracy", "AUC"],
        distillation_loss_fn=keras.losses.KLDivergence(),

    )
    # Train model
    history = distiller_fold.fit(
        X_train_fold, y_train_fold,
        validation_data=(X_val_fold, y_val_fold),
        batch_size=16,
        epochs=50,
        verbose=1
    )

    # Save result of each fold
    results = distiller_fold.evaluate(X_val_fold, y_val_fold)
    all_metrics.append(results)
    y_pred_fold = student_fold.predict(X_val_fold)
    y_pred_classes = np.argmax(y_pred_fold, axis=1)
    y_true_classes = np.argmax(y_val_fold, axis=1)

    # confusion matrix and save TP, TN, FP, FN
    cm = confusion_matrix(y_true_classes, y_pred_classes)
    tn, fp, fn, tp = cm.ravel()
    confusion_matrices.append({"TP": tp, "TN": tn, "FP": fp, "FN": fn})
    print(f"Fold {fold + 1} - TP: {tp}, TN: {tn}, FP: {fp}, FN: {fn}")

# Average the results from the folds
avg_metrics = np.mean(all_metrics, axis=0)
print(f"\nAverage Metrics over {k}-Fold Cross Validation:")
for metric_name, avg_metric in zip(distiller_fold.metrics_names, avg_metrics):
    print(f"{metric_name}: {avg_metric:.4f}")

# Show TP, TN, FP, FN for each fold
for fold, cm in enumerate(confusion_matrices):
    print(f"Fold {fold + 1} - TP: {cm['TP']}, TN: {cm['TN']}, FP: {cm['FP']}, FN: {cm['FN']}")
# save TP, TN, FP, FN into the file Excel
df_confusion = pd.DataFrame(confusion_matrices)
df_confusion.to_excel(path_result + "cv_Generics_KD2_Succi.xlsx", index=False)

print("\nConfusion matrix results saved to 'cv_Generics_KD2_Succi.xlsx'")

Training on fold 1/10...


/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)


Epoch 1/50


/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


802/802 ━━━━━━━━━━━━━━━━━━━━ 20s 18ms/step - AUC: 0.5933 - categorical_accuracy: 0.5737 - loss: 0.7788
Epoch 2/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - AUC: 0.6616 - categorical_accuracy: 0.6337 - loss: 0.6705
Epoch 3/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - AUC: 0.6831 - categorical_accuracy: 0.6545 - loss: 0.6466
Epoch 4/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 18s 13ms/step - AUC: 0.6816 - categorical_accuracy: 0.6539 - loss: 0.6440
Epoch 5/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 11s 14ms/step - AUC: 0.6881 - categorical_accuracy: 0.6567 - loss: 0.6400
Epoch 6/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 19s 13ms/step - AUC: 0.7081 - categorical_accuracy: 0.6668 - loss: 0.6262
Epoch 7/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 9s 11ms/step - AUC: 0.7257 - categorical_accuracy: 0.6807 - loss: 0.6128
Epoch 8/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 10s 12ms/step - AUC: 0.7249 - categorical_accuracy: 0.6756 - loss: 0.6131
Epoch 9/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 10s 13ms/step - AUC: 0.7254 - categorical_accuracy: 0.6703 -

...

802/802 ━━━━━━━━━━━━━━━━━━━━ 50s 48ms/step - AUC: 0.6664 - categorical_accuracy: 0.6504 - distill_loss1: 0.0309 - distill_loss2: 0.0849 - loss: 0.5000 - student_loss: 0.6361 - val_AUC: 0.7303 - val_categorical_accuracy: 0.6758 - val_loss: 0.5000 - val_student_loss: 0.3928
Epoch 2/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 42s 50ms/step - AUC: 0.7176 - categorical_accuracy: 0.6648 - distill_loss1: 0.0257 - distill_loss2: 0.0791 - loss: 0.5000 - student_loss: 0.6080 - val_AUC: 0.7533 - val_categorical_accuracy: 0.6800 - val_loss: 0.5000 - val_student_loss: 0.1271
Epoch 3/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 37s 47ms/step - AUC: 0.7707 - categorical_accuracy: 0.6966 - distill_loss1: 0.0184 - distill_loss2: 0.0709 - loss: 0.5000 - student_loss: 0.5661 - val_AUC: 0.8012 - val_categorical_accuracy: 0.7228 - val_loss: 0.5000 - val_student_loss: 0.1095
Epoch 4/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 46s 53ms/step - AUC: 0.7952 - categorical_accuracy: 0.7104 - distill_loss1: 0.0145 - distill_loss2: 0.0660 - loss: 0.50

/usr/local/lib/python3.10/dist-packages/keras/src/layers/core/embedding.py:90: UserWarning: Argument `input_length` is deprecated. Just remove it.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/keras/src/layers/rnn/bidirectional.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)
/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/nn.py:567: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


802/802 ━━━━━━━━━━━━━━━━━━━━ 18s 15ms/step - AUC: 0.6315 - categorical_accuracy: 0.6000 - loss: 0.7278
Epoch 2/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 22s 16ms/step - AUC: 0.6616 - categorical_accuracy: 0.6342 - loss: 0.6652
Epoch 3/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 19s 15ms/step - AUC: 0.6784 - categorical_accuracy: 0.6526 - loss: 0.6483
Epoch 4/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - AUC: 0.6846 - categorical_accuracy: 0.6570 - loss: 0.6421
Epoch 5/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 21s 16ms/step - AUC: 0.7016 - categorical_accuracy: 0.6693 - loss: 0.6289
Epoch 6/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - AUC: 0.7024 - categorical_accuracy: 0.6628 - loss: 0.6289
Epoch 7/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - AUC: 0.7184 - categorical_accuracy: 0.6645 - loss: 0.6187
Epoch 8/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 13s 16ms/step - AUC: 0.7224 - categorical_accuracy: 0.6662 - loss: 0.6154
Epoch 9/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 20s 16ms/step - AUC: 0.7244 - categorical_accuracy: 0.6681 

/usr/local/lib/python3.10/dist-packages/keras/src/backend/tensorflow/trainer.py:582: UserWarning: `model.compiled_metrics()` is deprecated. Instead, use e.g.:
```
for metric in self.metrics:
    metric.update_state(y, y_pred)
```

  return self._compiled_metrics_update_state(


802/802 ━━━━━━━━━━━━━━━━━━━━ 52s 49ms/step - AUC: 0.6785 - categorical_accuracy: 0.6649 - distill_loss1: 0.0270 - distill_loss2: 0.0834 - loss: 0.5000 - student_loss: 0.6321 - val_AUC: 0.7340 - val_categorical_accuracy: 0.6800 - val_loss: 0.5000 - val_student_loss: 0.5733
Epoch 2/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 39s 47ms/step - AUC: 0.7361 - categorical_accuracy: 0.6656 - distill_loss1: 0.0206 - distill_loss2: 0.0750 - loss: 0.5000 - student_loss: 0.5923 - val_AUC: 0.7825 - val_categorical_accuracy: 0.6926 - val_loss: 0.5000 - val_student_loss: 0.3724
Epoch 3/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 43s 50ms/step - AUC: 0.7765 - categorical_accuracy: 0.7042 - distill_loss1: 0.0151 - distill_loss2: 0.0678 - loss: 0.5000 - student_loss: 0.5549 - val_AUC: 0.8135 - val_categorical_accuracy: 0.7277 - val_loss: 0.5000 - val_student_loss: 0.3847
Epoch 4/50
802/802 ━━━━━━━━━━━━━━━━━━━━ 38s 47ms/step - AUC: 0.7955 - categorical_accuracy: 0.7162 - distill_loss1: 0.0125 - distill_loss2: 0.0642 - loss: 0.50